In [2]:
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

2025-06-30 13:52:37.334257: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751291557.493746      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751291557.539468      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
path=['/kaggle/input/cicids2017/Benign-Monday-no-metadata.parquet',
      '/kaggle/input/cicids2017/Botnet-Friday-no-metadata.parquet',
      '/kaggle/input/cicids2017/Bruteforce-Tuesday-no-metadata.parquet',
      '/kaggle/input/cicids2017/DDoS-Friday-no-metadata.parquet',
      '/kaggle/input/cicids2017/DoS-Wednesday-no-metadata.parquet',
      '/kaggle/input/cicids2017/Infiltration-Thursday-no-metadata.parquet',
      '/kaggle/input/cicids2017/Portscan-Friday-no-metadata.parquet',
      '/kaggle/input/cicids2017/WebAttacks-Thursday-no-metadata.parquet']
dfs=[]
for filenames in path:
     data=pd.read_parquet(filenames)
     dfs.append(data)
    

In [4]:
data=pd.concat(dfs,ignore_index=True)

In [5]:
data.head()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6,4,2,0,12,0,6,6,6.00000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
1,6,1,2,0,12,0,6,6,6.00000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
2,6,3,2,0,12,0,6,6,6.00000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
3,6,1,2,0,12,0,6,6,6.00000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
4,6,609,7,4,484,414,233,0,69.14286,111.967896,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign


In [6]:
data.isnull().sum()

Protocol                    0
Flow Duration               0
Total Fwd Packets           0
Total Backward Packets      0
Fwd Packets Length Total    0
                           ..
Idle Mean                   0
Idle Std                    0
Idle Max                    0
Idle Min                    0
Label                       0
Length: 78, dtype: int64

In [7]:
label_encoder=LabelEncoder()
data['Label']=label_encoder.fit_transform(data['Label'])

In [9]:
label_encoder.classes_

array(['Benign', 'Bot', 'DDoS', 'DoS GoldenEye', 'DoS Hulk',
       'DoS Slowhttptest', 'DoS slowloris', 'FTP-Patator', 'Heartbleed',
       'Infiltration', 'PortScan', 'SSH-Patator',
       'Web Attack � Brute Force', 'Web Attack � Sql Injection',
       'Web Attack � XSS'], dtype=object)

In [10]:
import joblib

joblib.dump(label_encoder,"label_encoder.pkl")

['label_encoder.pkl']

In [11]:
label_mapping=dict(zip(label_encoder.classes_,label_encoder.transform(label_encoder.classes_)))

In [12]:
label_mapping

{'Benign': 0,
 'Bot': 1,
 'DDoS': 2,
 'DoS GoldenEye': 3,
 'DoS Hulk': 4,
 'DoS Slowhttptest': 5,
 'DoS slowloris': 6,
 'FTP-Patator': 7,
 'Heartbleed': 8,
 'Infiltration': 9,
 'PortScan': 10,
 'SSH-Patator': 11,
 'Web Attack � Brute Force': 12,
 'Web Attack � Sql Injection': 13,
 'Web Attack � XSS': 14}

In [13]:
X=data.drop(columns='Label')
Y=data['Label']

In [14]:
X_train,X_test_,Y_train,Y_test_=train_test_split(X,Y,test_size=0.30,stratify=Y,random_state=42)
X_test,X_val,Y_test,Y_val=train_test_split(X_test_,Y_test_,test_size=0.15,stratify=Y_test_,random_state=42)

In [15]:
X_train.shape

(1619667, 77)

In [16]:
from sklearn.preprocessing import StandardScaler

standard_scaler=StandardScaler()
X_train_scaled=standard_scaler.fit_transform(X_train)
X_test_scaled=standard_scaler.transform(X_test)
X_val_scaled=standard_scaler.transform(X_val)

In [17]:
from sklearn.decomposition import PCA

In [18]:
import joblib
joblib.dump('standard_scaler','cic-ids-2017-standard-scaler.pkl')

['cic-ids-2017-standard-scaler.pkl']

In [19]:
X_train_scaled_reshaped=X_train_scaled.reshape((X_train_scaled.shape[0],X_train_scaled.shape[1],1))
X_val_scaled_reshaped=X_val_scaled.reshape((X_val_scaled.shape[0],X_val_scaled.shape[1],1))
X_test_scaled_reshaped=X_test_scaled.reshape((X_test_scaled.shape[0],X_test_scaled.shape[1],1))

In [20]:
input_shape=X_train_scaled_reshaped.shape[1:]
input_shape

(77, 1)

In [21]:
num_classes=len(np.unique(Y_train))
num_classes

15

In [22]:
Y_train_cat=to_categorical(Y_train,num_classes=num_classes)
Y_val_cat=to_categorical(Y_val,num_classes=num_classes)
Y_test_cat=to_categorical(Y_test,num_classes=num_classes)


In [23]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional,LSTM,Flatten,Dense,Dropout,Input,Conv1D,MaxPooling1D
input=Input(input_shape)

cnn1=Conv1D(filters=64,kernel_size=3,activation='relu')(input)
max1=MaxPooling1D(pool_size=2)(cnn1)
cnn2=Conv1D(filters=128,kernel_size=3,activation='relu')(max1)
max2=MaxPooling1D(pool_size=2)(cnn2)
cnn3=Conv1D(filters=256,kernel_size=3,activation='relu')(max2)
max3=MaxPooling1D(pool_size=2)(cnn3)
bidirlstm1=Bidirectional(LSTM(units=32,activation='relu',return_sequences=True))(max3)
bidirlstm2=Bidirectional(LSTM(units=64,activation='relu',return_sequences=True))(bidirlstm1)
bidirlstm3=Bidirectional(LSTM(units=128,activation='relu',return_sequences=True))(bidirlstm2)
bidirlstm4=Bidirectional(LSTM(units=256,activation='relu'))(bidirlstm3)
fl=Flatten()(bidirlstm4)
d1=Dense(units=128,activation='relu')(fl)
dr1=Dropout(0.2)(d1)
d2=Dense(units=256,activation='relu')(dr1)
dr2=Dropout(0.2)(d2)
d3=Dense(units=512,activation='relu')(dr2)
dr3=Dropout(0.2)(d3)
d4=Dense(units=1028,activation='relu')(dr3)
dr4=Dropout(0.2)(d4)
d2=Dense(units=num_classes,activation='softmax')(dr2)

model=Model(inputs=input,outputs=d2)
model.compile(loss='categorical_crossentropy',
            optimizer='adam',
             metrics=['Accuracy'])
model.fit(X_train_scaled_reshaped,Y_train_cat,validation_data=(X_val_scaled_reshaped,Y_val_cat),epochs=10,batch_size=1024)

I0000 00:00:1751291665.682465      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1751291665.683143      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Epoch 1/10


I0000 00:00:1751291683.556498     108 service.cc:148] XLA service 0xb1b6f800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751291683.556875     108 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1751291683.556925     108 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1751291685.306482     108 cuda_dnn.cc:529] Loaded cuDNN version 90300


   2/1582 ━━━━━━━━━━━━━━━━━━━━ 2:05 80ms/step - Accuracy: 0.2166 - loss: 2.7049   

I0000 00:00:1751291693.503943     108 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1582/1582 ━━━━━━━━━━━━━━━━━━━━ 114s 56ms/step - Accuracy: 0.9392 - loss: 0.2950 - val_Accuracy: 0.9862 - val_loss: 0.0453
Epoch 2/10
1582/1582 ━━━━━━━━━━━━━━━━━━━━ 79s 50ms/step - Accuracy: 0.9851 - loss: 0.0525 - val_Accuracy: 0.9826 - val_loss: 0.0550
Epoch 3/10
1582/1582 ━━━━━━━━━━━━━━━━━━━━ 78s 50ms/step - Accuracy: 0.9874 - loss: 0.0414 - val_Accuracy: 0.9898 - val_loss: 0.0308
Epoch 4/10
1582/1582 ━━━━━━━━━━━━━━━━━━━━ 78s 49ms/step - Accuracy: 0.9891 - loss: 0.0336 - val_Accuracy: 0.9893 - val_loss: 0.0299
Epoch 5/10
1582/1582 ━━━━━━━━━━━━━━━━━━━━ 78s 49ms/step - Accuracy: 0.9900 - loss: 0.0299 - val_Accuracy: 0.9885 - val_loss: 0.0385
Epoch 6/10
1582/1582 ━━━━━━━━━━━━━━━━━━━━ 78s 49ms/step - Accuracy: 0.9889 - loss: 0.0340 - val_Accuracy: 0.9907 - val_loss: 0.0271
Epoch 7/10
1582/1582 ━━━━━━━━━━━━━━━━━━━━ 78s 50ms/step - Accuracy: 0.9904 - loss: 0.0278 - val_Accuracy: 0.9912 - val_loss: 0.0261
Epoch 8/10
1582/1582 ━━━━━━━━━━━━━━━━━━━━ 78s 49ms/step - Accuracy: 0.9911 - loss: 0.0

In [24]:
attack_types=label_encoder.classes_

In [25]:


# Step 1: Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled_reshaped, Y_test_cat)
print(f"loss and accuracy are : {loss:.4f} and {accuracy:.4f}")

# Step 2: Prepare labels
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

Y_test_true = np.argmax(Y_test_cat, axis=1)
prediction = model.predict(X_test_scaled_reshaped)
Y_test_pred = np.argmax(prediction, axis=1)

# Step 3: Classification report
cls_report = classification_report(Y_test_true, Y_test_pred, target_names=attack_types, zero_division=0)
print(cls_report)

# Step 4: Metrics
f1 = f1_score(Y_test_true, Y_test_pred, average='weighted', zero_division=0)
precision = precision_score(Y_test_true, Y_test_pred, average='weighted', zero_division=0)
recall = recall_score(Y_test_true, Y_test_pred, average='weighted', zero_division=0)

print(f"Weighted F1-score: {f1:.4f}")
print(f"Weighted Precision: {precision:.4f}")
print(f"Weighted Recall: {recall:.4f}")


18439/18439 ━━━━━━━━━━━━━━━━━━━━ 72s 4ms/step - Accuracy: 0.9917 - loss: 0.0265
loss and accuracy are : 0.0260 and 0.9917
18439/18439 ━━━━━━━━━━━━━━━━━━━━ 61s 3ms/step
                            precision    recall  f1-score   support

                    Benign       0.99      1.00      1.00    504215
                       Bot       0.80      0.50      0.62       366
                      DDoS       1.00      0.98      0.99     32643
             DoS GoldenEye       0.99      0.96      0.97      2623
                  DoS Hulk       0.96      0.97      0.97     44076
          DoS Slowhttptest       0.87      0.95      0.91      1333
             DoS slowloris       0.95      0.98      0.96      1374
               FTP-Patator       1.00      0.97      0.98      1512
                Heartbleed       0.00      0.00      0.00         3
              Infiltration       0.00      0.00      0.00         9
                  PortScan       0.96      0.80      0.87       499
               

In [ ]:
model.save('cic-ids-2017.keras')

In [ ]:
import tensorflow as tf
import ker
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)

In [ ]:
!python --version
